In [11]:
import nidaqmx
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import os
import time
from nidaqmx.constants import AcquisitionType as AcqType
from nidaqmx.constants import TerminalConfiguration as TermConfig
from time import sleep
import qcodes as qc
import qcodes.instrument_drivers.QDevil.QDevil_QDAC as QDac
from qcodes.instrument_drivers.QDevil.QDevil_QDAC import Mode
from datetime import datetime
import h5py

def plot_Id_Vd(i):
    plt.figure(1, figsize=[7,5])
    plt.plot(Vd, Id[i,:], '-*')
    plt.xlabel('Drain to Source Voltage (V)')
    plt.ylabel('Drain to Source Current (nA)')
    plt.legend(Vg[:(i+1)])
    
def plot_Id_Vg(i):
    plt.figure(2, figsize=[7,5])
    plt.plot(Vg[:(i+1)], Id[:(i+1),:], '-*')
    plt.xlabel('Gate Voltage (V)')
    plt.ylabel('Drain to Source Current (nA)')
    plt.legend(Vd)
    


# gain of the preamps
gain = 1e6

# Data collection with NIDAQ at maximum sampling rate
V_range = 10 # in volts
rs_acqu = 1e6 # in Hz
rs_desired = 60 # in Hz

t_acqu = 1/rs_desired # n(=1) period of desired cycle
ns_acqu = int(rs_acqu*t_acqu)
t = np.arange(start=0, stop=t_acqu, step=1/rs_acqu)


#QDAC
qdac = QDac.QDac(name='qdac', address='ASRL5::INSTR', update_currents=False)
print("Number of channels: ", qdac.num_chans)
# voltage_slope = 1 # 0.01 V/sec
# qdac.ch01.slope(voltage_slope)
# qdac.ch02.slope(voltage_slope)

# Number of data points for the 2D sweeps
Nd = 10
Ng = 1

Vd = np.linspace(start = -10e-3, stop = +10e-3, num = Nd, endpoint=True)
Vg = np.linspace(start = 0, stop = 1, num = Ng, endpoint = True)
Id = np.zeros((Ng, Nd)) # currents and two voltages
Ig = np.zeros((Ng, Nd)) # recording gate leakage
Id_leak = np.zeros((Ng, Nd))

start_time = time.time() # to check how long it takes

%matplotlib qt
for i in np.arange(Ng):
    qdac.ch01.v.set(Vg[i])
    print('Gate voltage: ', Vg[i])
    sleep(0.1)
    for j in np.arange(Nd):
        # set the appropriate voltage
        qdac.ch02.v.set(Vd[j])
        print('Drain voltage: ', Vd[j])
        
        # Ig[i,j] = qdac.ch01.i.get()
        
        sleep(.1) # 1ms wait for voltage stabilization
        
        with nidaqmx.Task() as acqu_task:
            # Set acquisition channels and specs
            acqu_task.ai_channels.add_ai_voltage_chan("Dev2/ai1",terminal_config=TermConfig.DIFFERENTIAL,min_val=-V_range,max_val=+V_range)
            acqu_task.timing.cfg_samp_clk_timing(rs_acqu,sample_mode=AcqType.FINITE,samps_per_chan=ns_acqu)

            # Start acquisition
            acqu_task.start()

            # Collect the data measured
            current_i = acqu_task.read(number_of_samples_per_channel=ns_acqu)
            current_i = 1e9*np.array(current_i)/gain # voltage to actual current (in nA) conversion

            # Stop acquisition
            acqu_task.stop()
            
            # Assign to current array
            Id[i,j] = np.average(current_i)
            Ig[i, j] = qdac.ch01.i.get() #collecting gate leakage current
            Id_leak[i, j]= qdac.ch02.i.get()
    plot_Id_Vd(i)
    plt.pause(0.001)
    plot_Id_Vg(i)
    plt.pause(0.001)
            

stop_time = time.time()
print('Time taken for data collection: ', stop_time-start_time, ' (sec)')

#resetting all voltages
qdac.ch01.v.set(0)
qdac.ch02.v.set(0)

qdac.close()

for i in range(len(Vd)):
    plt.figure(5, figsize=[10,6])
    plt.plot(Vg, Ig[:,i],'-*', label=Vd[i])
    plt.xlabel('Gate Voltage (V)')
    plt.ylabel('Leakage Current (A)')
    plt.legend()

for i in range(len(Vg)):
    plt.figure(7, figsize=[10,6])
    plt.plot(Vd, Id_leak[i,:],'-*', label=Vg[i])
    plt.xlabel('Drain Voltage (V)')
    plt.ylabel('Leakage Current (A)')
    plt.legend()

save_data = True

path_directory = 'C:/Users/Measurement2/OneDrive/GroupShared/Data/QSim/20201130/'
meta_text = np.array(time.ctime()+'. Device 7 (BB remapped, no series resistor)) is connected.', dtype='S')

if save_data==True:
    # reading the index number
    file_num = open(path_directory+"filenum.txt","r")
    index = file_num.read()
    file_num.close()
    
    # saving the data into hdf5 file
    f = h5py.File(path_directory+'dat'+index+'.hdf5','w')
    f.create_dataset('data/Id', data=Id)
    f.create_dataset('data/Ig', data=Ig)
    f.create_dataset('data/Id_leak', data=Id_leak)
    f.create_dataset('variable/Vd', data=Vd)
    f.create_dataset('variable/Vg', data=Vg)
    f.create_dataset('metadata', data=[meta_text])
    f.close()
    print('Data is saved to dat'+index+'.hdf5 file.')
    
    # updating the index number
    file_num = open(path_directory+"filenum.txt","w")
    file_num.write(str(int(index)+1))
    file_num.close()

Number of channels:  24
Gate voltage:  0.0
Drain voltage:  -0.01
Drain voltage:  -0.0077777777777777776
Drain voltage:  -0.005555555555555556
Drain voltage:  -0.003333333333333334
Drain voltage:  -0.0011111111111111113
Drain voltage:  0.0011111111111111113
Drain voltage:  0.0033333333333333322
Drain voltage:  0.005555555555555555
Drain voltage:  0.0077777777777777776
Drain voltage:  0.01
Time taken for data collection:  8.59520959854126  (sec)
Data is saved to dat23.hdf5 file.


In [48]:
for i in range(len(Vd)):
    plt.figure(5, figsize=[10,6])
    plt.plot(Vg, Ig[:,i],'-*', label=Vd[i])
    plt.xlabel('Gate Voltage (V)')
    plt.ylabel('Leakage Current (A)')
    plt.legend()

In [8]:
qdac.close()

In [65]:
qdac = QDac.QDac(name='qdac', address='ASRL5::INSTR', update_currents=False)
qdac.print_overview(update_currents=False)
qdac.close()

Channel 1 
    Voltage: 0.0 (V).
    Current: None (A).
    Mode: V range low / I range low.
    Slope: Inf (V/s).

Channel 2 
    Voltage: 0.0 (V).
    Current: None (A).
    Mode: V range low / I range low.
    Slope: Inf (V/s).

Channel 3 
    Voltage: -2e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 4 
    Voltage: 1e-06 (V).
    Current: None (A).
    Mode: V range low / I range low.
    Slope: Inf (V/s).

Channel 5 
    Voltage: -9e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 6 
    Voltage: 9e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 7 
    Voltage: 8e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 8 
    Voltage: 7e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 9 
    Voltage: -1e-06 (V).
    Current:

In [25]:
import h5py

def plot1D_Id_Vd(i):
    plt.figure(3, figsize=[10,6])
    plt.plot(Vd, Id[i,:],'-*' , label=Vg[i])
    plt.xlabel('Drain to Source Voltage (V)')
    plt.ylabel('Drain to Source Current (nA)')
    plt.legend()

def plot1D_Id_Vg(i):
    plt.figure(4, figsize=[10,6])
    plt.plot(Vg, Id[:,i],'-*', label=Vd[i])
    plt.xlabel('Gate Voltage (V)')
    plt.ylabel('Drain to Source Current (nA)')
    plt.legend()
    
%matplotlib qt

f = h5py.File('C:/Users/Measurement2/OneDrive/GroupShared/Data/QSim/20201130/dat6.hdf5','r')
Id = np.array(f['data/Id'])
Vg = np.array(f['variable/Vg'])
Vd = np.array(f['variable/Vd'])
f.close()

for i in [0, 2]:
    plot1D_Id_Vd(i)

for i in [0, 3]:
    plot1D_Id_Vg(i)

In [ ]:
f = h5py.File('C:/Users/Measurement2/OneDrive/GroupShared/Data/QSim/20201130/dat18.hdf5','a')
f.create_dataset
f.close()

In [15]:
%whos

Variable         Type                          Data/Info
--------------------------------------------------------
AcqType          EnumMeta                      <enum 'AcquisitionType'>
Id               ndarray                       3x5: 15 elems, type `float64`, 120 bytes
Ig               ndarray                       3x5: 15 elems, type `float64`, 120 bytes
Mode             EnumMeta                      <enum 'Mode'>
Nd               int                           5
Ng               int                           3
QDac             module                        <module 'qcodes.instrumen<...>\QDevil\\QDevil_QDAC.py'>
TermConfig       EnumMeta                      <enum 'TerminalConfiguration'>
V_range          int                           10
Vd               ndarray                       5: 5 elems, type `float64`, 40 bytes
Vg               ndarray                       3: 3 elems, type `float64`, 24 bytes
acqu_task        Task                          <object with id 2553710632200 (st

In [19]:
plt.figure()
plt.plot(t[0:-1], current_i)

In [24]:
plt.close()

In [38]:

plt.figure(5, figsize=[10,6])
plt.plot(Vg, Ig[:,i],'-*', label=Vd[i])
plt.xlabel('Gate Voltage (V)')
plt.ylabel('Leakage Current (A)')
plt.legend()

In [43]:
plt.figure(6, figsize=[10,6])
plt.plot(Vd, Id_leak[i,:],'-*', label=Vg[i])
plt.xlabel('Drain Voltage (V)')
plt.ylabel('Leakage Current (A)')
plt.legend()

In [61]:
qdac = QDac.QDac(name='qdac', address='ASRL5::INSTR', update_currents=False)
qdac.ch02.mode(Mode.vlow_ilow)
qdac.print_overview(update_currents=False)
qdac.close()

Channel 1 
    Voltage: 0.0 (V).
    Current: None (A).
    Mode: V range low / I range low.
    Slope: Inf (V/s).

Channel 2 
    Voltage: 7e-06 (V).
    Current: None (A).
    Mode: V range low / I range low.
    Slope: Inf (V/s).

Channel 3 
    Voltage: -2e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 4 
    Voltage: 1e-06 (V).
    Current: None (A).
    Mode: V range low / I range low.
    Slope: Inf (V/s).

Channel 5 
    Voltage: -9e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 6 
    Voltage: 9e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 7 
    Voltage: 8e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 8 
    Voltage: 7e-06 (V).
    Current: None (A).
    Mode: V range high / I range high.
    Slope: Inf (V/s).

Channel 9 
    Voltage: -1e-06 (V).
    Curren

In [47]:
for i in range(len(Vg)):
    plt.figure(7, figsize=[10,6])
    plt.plot(Vd, Id_leak[i,:],'-*', label=Vg[i])
    plt.xlabel('Drain Voltage (V)')
    plt.ylabel('Leakage Current (A)')
    plt.legend()

In [55]:
10e-3/((1.88+3.68)*1e-6)

1798.5611510791366

In [66]:
import h5py
f = h5py.File('C:/Users/Measurement2/OneDrive/GroupShared/Data/QSim/20201130/dat19.hdf5','r')

In [67]:
f.keys()

<KeysViewHDF5 ['data', 'metadata', 'variable']>

In [68]:
f.visit(print)

data
data/Id
data/Id_leak
data/Ig
metadata
variable
variable/Vd
variable/Vg


In [69]:
print(f['metadata'][:])

[b'Mon Nov 30 11:26:17 2020. Device 7 (BB remapped, no series resistor)) is connected.']


In [70]:
f.close()